In [2]:
# Library
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [31]:
words_data = pd.read_csv('./Data/Preprocess_Data/final_words.csv')
abstract_data = pd.read_csv('./Data/Preprocess_Data/Abstract_preprocessing.csv')

In [32]:
abstract_data = abstract_data[['Year','Abstract2']]

In [33]:
abstract_data.head()

,Year,Abstract2
0,2013,a method for peer to peer streaming of video o...
1,2013,in this paper we discus the bacterial network ...
2,2013,this article treat a digital humanity work in ...
3,2013,this work describes preliminary step towards n...
4,2013,goal extraction in learning by demonstration i...


In [34]:
words_data[words_data['word2'] == '802.11']

,word,word2,length
8887,802.11,802.11,6


In [47]:
# 'Abstract2' 칼럼의 값을 변경하여 'Abstract' 칼럼을 생성합니다.
abstract_data['Abstract2'] = abstract_data['Abstract2'].str.replace('80211', '802.11')

In [48]:
abstract_data

,Year,Abstract2
0,2013,a method for peer to peer streaming of video o...
1,2013,in this paper we discus the bacterial network ...
2,2013,this article treat a digital humanity work in ...
3,2013,this work describes preliminary step towards n...
4,2013,goal extraction in learning by demonstration i...
...,...,...
2267158,2021,human centered development of information syst...
2267159,2021,the computing device in cloud or fog data cent...
2267160,2021,mobile technology are becoming more and more a...
2267161,2021,development of intelligent system with the pur...


In [49]:
words_data.head()

,word,word2,length
0,bidirectional encoder representations from tra...,bidirectional_encoder_representations_from_tra...,55
1,bidirectional encoder representations from tra...,bidirectional_encoder_representations_from_tra...,54
2,deep convolutional generative adversarial network,deep_convolutional_generative_adversarial_network,49
3,contrast limited adaptive histogram equalization,contrast_limited_adaptive_histogram_equalization,48
4,covariance matrix adaptation evolution strategy,covariance_matrix_adaptation_evolution_strategy,47


- 단어 데이터 전처리 (x가 아닌 것들만 남기기)
- 여기서는 이미 진행한 부분이여서 생략

In [3]:
words_data = words_data[words_data['제거']!='x']
words_data.reset_index(drop=True, inplace=True)
words_data.drop('제거',axis=1, inplace=True)
len(words_data)

9187

In [4]:
# 양쪽 공백 제거
words_data['word'] = words_data['word'].str.strip()

In [5]:
# words_data에 공백을 언더바(_)로 대체
words_data['word2'] = 0
for i in range(len(words_data)):
    if len(str(words_data['word'][i]).split(' ')) >1:
        words_data['word2'][i] = words_data['word'][i].replace(' ','_')
    else:
        words_data['word2'][i] = words_data['word'][i]

In [6]:
words_data['length'] = 0
for i in range(len(words_data)):
    words_data['length'][i] = len(str(words_data['word'][i]))

In [7]:
# length에 따라 정렬 (변환이 제대로 되지 않는 것 방지)
words_data.sort_values(by='length', ascending=False, inplace=True)
words_data.reset_index(drop=True, inplace=True)
words_data.head()

,word,count,word2,length
0,bidirectional encoder representations from tra...,24,bidirectional_encoder_representations_from_tra...,54
1,deep convolutional generative adversarial network,31,deep_convolutional_generative_adversarial_network,49
2,contrast limited adaptive histogram equalization,32,contrast_limited_adaptive_histogram_equalization,48
3,covariance matrix adaptation evolution strategy,25,covariance_matrix_adaptation_evolution_strategy,47
4,ciphertext policy attribute based encryption,69,ciphertext_policy_attribute_based_encryption,44


In [8]:
# 중복되는 값 중 첫번째만 남기고 제거
words_data.drop_duplicates(subset='word2', keep='first', inplace=True)

In [9]:
len(words_data)

9155

- 여기서부터 실행

In [50]:
# words 칼럼의 단어를 words2 칼럼의 단어로 대체
abstract_data['Abstract3'] = ""

words_dict = dict(zip(words_data['word'], words_data['word2']))

for k in tqdm(range(len(abstract_data))):
    abstract = str(abstract_data['Abstract2'][k])
    replaced_abstract = abstract

    for word, word2 in words_dict.items():
        replaced_abstract = replaced_abstract.replace(word, word2)

    abstract_data['Abstract3'][k] = replaced_abstract

100%|██████████████████████████████| 2267163/2267163 [4:07:55<00:00, 152.41it/s]


In [52]:
new_abstract = abstract_data[['Year','Abstract3']]
new_abstract.to_csv('Data/Preprocess_Data/Abstract3.csv', encoding='utf-8-sig', index=True)

In [58]:
new_abstract[new_abstract['Abstract3'].str.contains('machine_learning')].iloc[0]['Abstract3']

'given a connected undirected graph whose edge are labelled the minimum labelling spanning tree problem seek a spanning tree whose edge have the smallest number of distinct label in recent work the mlst problem ha been shown to be np hard and some effective heuristic have been proposed and analysed in this paper we present preliminary result of a currently on going project regarding the implementation of an intelligent optimization algorithm to solve the mlst problem this algorithm is obtained by the basic variable neighbourhood search heuristic with the integration of other complement from machine_learning statistic and experimental algorithmics in order to produce high quality performance and to completely automate the resulting optimization strategy © 2013 springer verlag'

- 해당되는 단어가 있으면 연도붙여주기 (deep -> 13_deep)

In [10]:
new_abstract = pd.read_csv('./Data/Preprocess_Data/Abstract3.csv')

In [59]:
t = new_abstract.copy()

In [60]:
t.head()

,Year,Abstract3
0,2013,a method for peer to peer streaming of video o...
1,2013,in this paper we discus the bacterial network ...
2,2013,this article treat a digital humanity work in ...
3,2013,this work describes preliminary step towards n...
4,2013,goal extraction in learning by demonstration i...


In [61]:
# year 칼럼의 뒤의 두 자리 추출
t['year_suffix'] = t['Year'].astype(str).str[-2:]
# list 칼럼의 단어들을 분리하여 리스트로 변환
word_lists = t['Abstract3'].str.split()

In [62]:
# 단어 집합을 집합(Set)으로 변환하여 탐색 시간을 단축
words_subset = set(words_data['word2'])

# 정규식 패턴을 미리 생성하여 반복문 내에서 재사용
pattern = re.compile(r'\b({})\b'.format('|'.join(map(re.escape, words_subset))))

In [63]:
len(words_subset)

9121

In [64]:
# 단어가 해당되면 연도까지 붙여주기
new_word_lists = []  # 매칭된 단어에 suffix가 추가된 리스트

for words, suffix in tqdm(zip(word_lists, t['year_suffix']), total=len(t)):
    new_words = []
    for word in words:
        if pattern.match(word):
            word = str(suffix) + '_' + word

            new_words.append(word)
        else:
            new_words.append(word)
    new_word_lists.append(new_words)

100%|██████████████████████████████| 2267163/2267163 [3:16:59<00:00, 191.82it/s]


In [65]:
new_word_strings = [' '.join(word_list) for word_list in new_word_lists]

In [66]:
t['new_abstract'] = new_word_strings

In [67]:
t

,Year,Abstract3,year_suffix,new_abstract
0,2013,a method for peer to peer streaming of video o...,13,a method for peer to peer streaming of video o...
1,2013,in this paper we discus the bacterial network ...,13,in this paper we discus the bacterial network ...
2,2013,this article treat a digital humanity work in ...,13,this article treat a digital humanity work in ...
3,2013,this work describes preliminary step towards n...,13,this work describes preliminary step towards n...
4,2013,goal extraction in learning by demonstration i...,13,goal extraction in learning by 13_demonstratio...
...,...,...,...,...
2267158,2021,human centered development of information syst...,21,human centered development of information syst...
2267159,2021,the computing device in cloud or fog data_cent...,21,the computing device in 21_cloud or fog 21_dat...
2267160,2021,mobile technology are becoming more and more a...,21,mobile technology are becoming more and more a...
2267161,2021,development of intelligent system with the pur...,21,development of intelligent system with the pur...


- 기타 전처리

In [68]:
t.to_csv('연도붙인버전.csv', encoding='utf-8-sig', index=False)

In [69]:
# 정규표현식을 사용하여 's와 영어 알파벳, 숫자, 그리고 underscore('_') 이외의 문자 제거
t['new_abstract'] = t['new_abstract'].astype(str)
t['new_abstract'] = t['new_abstract'].apply(lambda x: re.sub(r"[^a-zA-Z0-9_\s]|'s\b", '', x))

In [70]:
# 공백이 2칸 이상이면 한칸으로 줄어듦
t['new_abstract'] = t['new_abstract'].apply(lambda x: re.sub(r'\s{2,}', ' ', x))

In [71]:
import logging
from tqdm import tqdm
tqdm.pandas()

t['new_abstract'] = t['new_abstract'].astype(str)

# 불용어 설정
stop_words = set(stopwords.words('english'))

# tokens 칼럼 : 불용어 제거 추가버전
chunk_size = 300000  # 처리할 청크 크기 설정

# 청크별로 데이터 처리
total_rows = len(t)
start = 0
end = chunk_size

pbar = tqdm(total=total_rows)  # 진행 상황 표시를 위한 tqdm 객체 생성

while start < total_rows:
    chunk_data = t['new_abstract'].iloc[start:end]
    t.loc[start:end, 'tokens'] = chunk_data.progress_apply(lambda x: [word for word in word_tokenize(x) if word not in stop_words])

    start += chunk_size
    end += chunk_size
    if end > total_rows:
        end = total_rows

    pbar.update(chunk_size)  # tqdm 객체 업데이트

pbar.close()  # tqdm 객체 종료

  8%|██▉                               | 25465/300000 [00:11<01:43, 2650.72it/s]


 17%|█████▉                            | 52339/300000 [00:21<01:33, 2646.66it/s]


 25%|████████▋                         | 76123/300000 [07:39<01:37, 2293.21it/s]


 34%|███████████▎                     | 103352/300000 [07:49<01:10, 2779.22it/s]


 43%|██████████████▎                  | 130426/300000 [07:59<01:03, 2684.58it/s]


...

 49%|████████████████▌                 | 81601/167163 [00:32<00:31, 2686.18it/s]


 65%|█████████████████████▎           | 108072/167163 [00:42<00:20, 2953.35it/s]


 80%|██████████████████████████▎      | 133416/167163 [00:52<00:14, 2344.47it/s]


 95%|███████████████████████████████▎ | 158676/167163 [01:02<00:03, 2521.08it/s]


100%|█████████████████████████████████| 167163/167163 [01:06<00:00, 2531.45it/s]
2400000it [31:35, 1266.37it/s]                                                  


In [72]:
t.head()

,Year,Abstract3,year_suffix,new_abstract,tokens
0,2013,a method for peer to peer streaming of video o...,13,a method for peer to peer streaming of video o...,"[method, peer, peer, streaming, video, demand,..."
1,2013,in this paper we discus the bacterial network ...,13,in this paper we discus the bacterial network ...,"[paper, discus, bacterial, network, 13_communi..."
2,2013,this article treat a digital humanity work in ...,13,this article treat a digital humanity work in ...,"[article, treat, digital, humanity, work, clas..."
3,2013,this work describes preliminary step towards n...,13,this work describes preliminary step towards n...,"[work, describes, preliminary, step, towards, ..."
4,2013,goal extraction in learning by demonstration i...,13,goal extraction in learning by 13_demonstratio...,"[goal, extraction, learning, 13_demonstration,..."


In [73]:
t['tokens'] = t['tokens'].apply(lambda x: ' '.join(x))

In [74]:
t['tokens']

0          method peer peer streaming video demand reside...
1          paper discus bacterial network 13_communicatio...
2          article treat digital humanity work classical ...
3          work describes preliminary step towards nano s...
4          goal extraction learning 13_demonstration comp...
                                 ...                        
2267158    human centered development information system ...
2267159    computing device 21_cloud fog 21_data_center r...
2267160    mobile technology becoming accepted used pedag...
2267161    development intelligent system pursuit detecti...
2267162    paper gauge utility general purpose open domai...
Name: tokens, Length: 2267163, dtype: object

In [75]:
t2 = t[['Year','tokens']]

In [76]:
t2.head()

,Year,tokens
0,2013,method peer peer streaming video demand reside...
1,2013,paper discus bacterial network 13_communicatio...
2,2013,article treat digital humanity work classical ...
3,2013,work describes preliminary step towards nano s...
4,2013,goal extraction learning 13_demonstration comp...


In [77]:
replacement_dict = {
    'ada_boost': 'adaboost',
    'zig_bee': 'zigbee',
    'optimisation': 'optimization',
    'neural_networks': 'neural_network',
    'audiovisual': 'audio_visual',
    'authorisation': 'authorization',
    'chatbots': 'chatbot',
    'conceptualisation': 'conceptualization'
}

def replace_words(text):
    for old_word, new_word in replacement_dict.items():
        text = text.replace(old_word, new_word)
    return text

tqdm.pandas()  # tqdm을 사용하기 위해 pandas에 연결

t2['tokens'] = t2['tokens'].progress_apply(replace_words)

100%|█████████████████████████████| 2267163/2267163 [00:11<00:00, 204949.59it/s]


In [78]:
t2.to_csv('전처리완료토큰.csv', encoding='utf-8-sig', index=False)

- 다음 코드 : 그리디 알고리즘